##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Recommenders: Quickstart

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/recommenders/quickstart"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/recommenders/blob/main/docs/examples/quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/recommenders/blob/main/docs/examples/quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/recommenders/docs/examples/quickstart.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In this tutorial, we build a simple matrix factorization model using the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with TFRS. We can use this model to recommend movies for a given user.

### Import TFRS

First, install and import TFRS:

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [ ]:
from typing import Dict, Text

import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import pandas as pd

### Read the data

In [ ]:
# Ratings data.
user_ids = ["1", "2", "3", "4", "5"]
book_titles = ["Book A", "Book B", "Book C", "Book D", "Book E"]
ratings_data = {
    "user_id": ["1", "2", "3", "4", "5", "1", "2", "3", "4", "5"],
    "book_title": ["Book A", "Book B", "Book C", "Book D", "Book E", "Book B", "Book C", "Book D", "Book E", "Book A"],
    "rating": [5, 4, 3, 4, 5, 5, 4, 3, 4, 5]
}

ratings_df = pd.DataFrame(ratings_data)
books_df = pd.DataFrame({"book_title": book_titles})


Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [ ]:
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_df))
books = tf.data.Dataset.from_tensor_slices(dict(books_df))
vocabulary = list(books.map(lambda x: x["book_title"]).as_numpy_iterator())

### Define a model

We can define a TFRS model by inheriting from `tfrs.Model` and implementing the `compute_loss` method:

In [ ]:
class BookLensModel(tfrs.Model):
    def __init__(self, user_model, book_model, task):
        super().__init__()
        self.book_model: tf.keras.Model = book_model
        self.user_model: tf.keras.Model = user_model
        self.task: tf.keras.layers.Layer = task

    def compute_loss(self, features, training=False):
        user_embeddings = self.user_model(features["user_id"])
        positive_book_embeddings = self.book_model(features["book_title"])
        return self.task(user_embeddings, positive_book_embeddings)

user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(vocabulary=user_ids, mask_token=None),
    tf.keras.layers.Embedding(len(user_ids) + 1, 32)
])

book_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(vocabulary=vocabulary, mask_token=None),
    tf.keras.layers.Embedding(len(vocabulary) + 1, 32)
])

task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=books.batch(128).map(lambda x: book_model(x["book_title"])) # Extract "book_title" and apply book_model
    )
)

Define the two models and the retrieval task.

In [ ]:
model = BookLensModel(user_model, book_model, task)



### Fit and evaluate it.

Create the model, train it, and generate predictions:



In [ ]:
# Compile the model
model = BookLensModel(user_model, book_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train the model
model.fit(ratings.batch(4096), epochs=3)

# Set up retrieval using the trained representations
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    books.batch(100).map(lambda title: (title["book_title"], model.book_model(title["book_title"]))))
# Get some recommendations.  Set k to 5
_, titles = index(np.array(["1"]), k=5)
print(f"Top 3 recommendations for user 1: {titles[0, :3]}")